# Overview
The goal of this challenge is to predict future claims for health insurance enrollees. This will help the healthcare insurer adequately set the premiums.
The data consists of patient profile including patient age, sex ,chronic conditions and severity of each chronic condition, history of outpatient claims for past few years is also included.
The goal is to predict the total claims for the next year.

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import optuna
from scipy.stats import zscore
from sklearn.model_selection import KFold
# 1. Load the data
train_json_path = '/kaggle/input/healthcare-claims-prediction/patient_data_train.json'
test_json_path = '/kaggle/input/healthcare-claims-prediction/patient_data_test.json'
train_csv_path = '/kaggle/input/healthcare-claims-prediction/train.csv'
submission_csv_path = '/kaggle/input/healthcare-claims-prediction/sample_submission.csv'

# Load JSON files
with open(train_json_path, 'r') as f:
    train_json_data = json.load(f)

with open(test_json_path, 'r') as f:
    test_json_data = json.load(f)

# Load CSV files
train_csv = pd.read_csv(train_csv_path)
submission_csv = pd.read_csv(submission_csv_path)

# 2. Preprocess the data and add the severity column
def process_json_data(json_data):
    processed_data = []
    for patient in json_data:
        patient_id = patient['PatientID']
        sex = 1 if patient['Sex'] == 'M' else 0
        age = patient['Age']
        conditions = patient['Conditions']

        total_severity = sum(conditions.values())

        outpatient_costs = patient.get('Out patient costs', {})
        avg_cost = {year: outpatient_costs.get(str(year), 0) for year in range(2019, 2024)}
        
        data_row = {
            'PatientID': patient_id,
            'Sex': sex,
            'Age': age,
            'Conditions_AT': conditions.get('AT', 0),
            'Conditions_DB': conditions.get('DB', 0),
            'Conditions_HT': conditions.get('HT', 0),
            'Conditions_HD': conditions.get('HD', 0),
            'TotalSeverity': total_severity,
            'Outpatient_2019': avg_cost[2019],
            'Outpatient_2020': avg_cost[2020],
            'Outpatient_2021': avg_cost[2021],
            'Outpatient_2022': avg_cost[2022],
            'Outpatient_2023': avg_cost[2023]
        }
        processed_data.append(data_row)
    
    return pd.DataFrame(processed_data)

# Process both train and test data
train_df = process_json_data(train_json_data)
test_df = process_json_data(test_json_data)

# 3. Merge the train CSV with the processed train JSON data
train_merged = pd.merge(train_df, train_csv, on='PatientID')

# Drop 'PatientID' column since it's not needed for training
train_merged.drop(columns=['PatientID'], inplace=True)

# Outlier removal based on TotalClaims using z-score
train_merged['zscore'] = zscore(train_merged['TotalClaims'])
train_merged = train_merged[train_merged['zscore'].abs() < 3]

# 4. Split the Data
X = train_merged.drop(columns=['TotalClaims', 'zscore'])
y = train_merged['TotalClaims']
X_train_important, X_val_important, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Feature Engineering
def add_advanced_features(df):
    """
    Adds new features to the dataset for better feature representation.
    """
    if 'Outpatient_2023' in df.columns:
        df['Outpatient_2023_log'] = np.log1p(df['Outpatient_2023'])  # Log transformation
    
    if 'Age' in df.columns:
        df['Age_Squared'] = df['Age'] ** 2
        df['Age_Cubed'] = df['Age'] ** 3  # Cubic feature
    
    if 'TotalSeverity' in df.columns:
        df['Severity_Squared'] = df['TotalSeverity'] ** 2
        df['Severity_Cubed'] = df['TotalSeverity'] ** 3  # Cubic feature
    
    # Interaction features
    df['Age_Severity_Interaction'] = df['Age'] * df['TotalSeverity']
    
    return df

# Apply feature engineering to training, validation, and test sets
X_train_important = add_advanced_features(X_train_important)
X_val_important = add_advanced_features(X_val_important)
test_df = add_advanced_features(test_df)

# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_important)
X_val_scaled = scaler.transform(X_val_important)
test_df_scaled = scaler.transform(test_df[X_train_important.columns])

# 6. Hyperparameter Tuning with Bayesian Optimization using Optuna
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10.0),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)  # Regularization parameter
    }
    
    model = xgb.XGBRegressor(objective='reg:squarederror', **param)
    
    model.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)],
              early_stopping_rounds=50, verbose=False)
    
    y_pred = model.predict(X_val_scaled)
    mae = mean_absolute_error(y_val, y_pred)
    return mae

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=150)

best_params = study.best_params
print(f"Best Parameters (XGBoost): {best_params}")

# 7. Train Final Model with Best Hyperparameters
best_xgb = xgb.XGBRegressor(
    **best_params,
    objective='reg:squarederror',
    random_state=42
)

best_xgb.fit(
    X_train_scaled,
    y_train,
    eval_set=[(X_val_scaled, y_val)],
    early_stopping_rounds=50,
    verbose=True
)

# 8. Model Evaluation
y_val_pred_xgb = best_xgb.predict(X_val_scaled)
val_mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)
print(f"Validation MAE (XGBoost) after tuning: {val_mae_xgb:.2f}")


# Cross-Validation (Use KFold for continuous target variable)
cv = KFold(n_splits=10, shuffle=True, random_state=42)
cv_scores_xgb = cross_val_score(best_xgb, X_train_scaled, y_train, cv=cv, scoring='neg_mean_absolute_error')
mean_cv_score_xgb = -np.mean(cv_scores_xgb)
print(f"Mean Cross-Validation MAE (XGBoost): {mean_cv_score_xgb:.2f}")


# Calculate Validation Percentage Error for XGBoost
val_percentage_error_xgb = np.mean(np.abs((y_val - y_val_pred_xgb) / y_val)) * 100
print(f"Validation Percentage Error (XGBoost): {val_percentage_error_xgb:.2f}%")

# 9. Predict on the test set and prepare submission
test_predictions_xgb = best_xgb.predict(test_df_scaled)

submission_df_xgb = submission_csv.copy()
submission_df_xgb['TotalClaims'] = test_predictions_xgb

submission_filename = 'final_submission_xgb_optimized_13.csv'
submission_df_xgb.to_csv(submission_filename, index=False)
print(f"Submission file '{submission_filename}' created successfully!")



[I 2024-12-02 23:54:07,684] A new study created in memory with name: no-name-cd8a0cd8-388e-4a7d-86fe-770aa89c1412
/tmp/ipykernel_36/2129493574.py:120: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10.0),
/tmp/ipykernel_36/2129493574.py:121: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10.0),
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2024-1

Best Parameters (XGBoost): {'n_estimators': 2853, 'max_depth': 5, 'learning_rate': 0.0663639365643843, 'subsample': 0.8511448363257955, 'colsample_bytree': 0.589672372754236, 'reg_alpha': 2.5594894096220454e-05, 'reg_lambda': 0.8633110639839161, 'min_child_weight': 1}
[0]	validation_0-rmse:13012.22030
[1]	validation_0-rmse:12295.59776
[2]	validation_0-rmse:11639.86309
[3]	validation_0-rmse:11038.99397
[4]	validation_0-rmse:10495.96059
[5]	validation_0-rmse:9978.12047
[6]	validation_0-rmse:9485.06191
[7]	validation_0-rmse:9002.14533
[8]	validation_0-rmse:8562.17274
[9]	validation_0-rmse:8188.83192
[10]	validation_0-rmse:7801.66711
[11]	validation_0-rmse:7478.67438
[12]	validation_0-rmse:7152.27469
[13]	validation_0-rmse:6849.13419
[14]	validation_0-rmse:6600.47057
[15]	validation_0-rmse:6323.76678
[16]	validation_0-rmse:6075.59023
[17]	validation_0-rmse:5863.56123
[18]	validation_0-rmse:5667.71154
[19]	validation_0-rmse:5473.82845
[20]	validation_0-rmse:5280.97746
[21]	validation_0-rmse

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[35]	validation_0-rmse:3606.53635
[36]	validation_0-rmse:3544.13153
[37]	validation_0-rmse:3490.99412
[38]	validation_0-rmse:3430.05353
[39]	validation_0-rmse:3364.15870
[40]	validation_0-rmse:3294.07843
[41]	validation_0-rmse:3245.20882
[42]	validation_0-rmse:3206.89810
[43]	validation_0-rmse:3152.04049
[44]	validation_0-rmse:3121.30338
[45]	validation_0-rmse:3081.42719
[46]	validation_0-rmse:3052.04411
[47]	validation_0-rmse:3009.08624
[48]	validation_0-rmse:2973.54961
[49]	validation_0-rmse:2950.33275
[50]	validation_0-rmse:2924.61521
[51]	validation_0-rmse:2904.61905
[52]	validation_0-rmse:2871.27977
[53]	validation_0-rmse:2836.41796
[54]	validation_0-rmse:2820.41355
[55]	validation_0-rmse:2808.03665
[56]	validation_0-rmse:2793.46970
[57]	validation_0-rmse:2760.17993
[58]	validation_0-rmse:2745.29860
[59]	validation_0-rmse:2720.67761
[60]	validation_0-rmse:2704.97413
[61]	validation_0-rmse:2692.44327
[62]	validation_0-rmse:2650.00566
[63]	validation_0-rmse:2641.84666
[64]	validatio